<a href="https://colab.research.google.com/github/joudiabouseif/DLClus/blob/main/AvgW2V.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
 
 
uploaded = files.upload()

Saving df2.csv to df2.csv


In [2]:
import pandas as pd
#load the data
df=pd.read_csv("df2.csv")

In [3]:
df['keywords'] = df['keywords'].str.lower()

In [4]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = stopwords.words('english')
df['keywords'] = df['keywords'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
df.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,datasetName,keywords
0,USA_cars_datasets,unnamed: 0 price brand model year title status...
1,cars,manufacturer name model name transmission colo...
2,test-data,unnamed: 0 name location year kilometers drive...
3,vehicles,id url region region url price year manufactur...
4,zoo,animal name hair feathers eggs milk airborne a...


In [5]:
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
ps = PorterStemmer()
def stem_sentences(sentence):
    tokens = sentence.split()
    stemmed_tokens = [ps.stem(token) for token in tokens]
    return ' '.join(stemmed_tokens)

df['keywords'] = df['keywords'].apply(stem_sentences)
df.head()

,datasetName,keywords
0,USA_cars_datasets,unnamed: 0 price brand model year titl statu m...
1,cars,manufactur name model name transmiss color odo...
2,test-data,unnamed: 0 name locat year kilomet driven fuel...
3,vehicles,id url region region url price year manufactur...
4,zoo,anim name hair feather egg milk airborn aquat ...


In [6]:
# Train your own Word2Vec model using your own text corpus
i=0
list_of_sent=[]
for sent in df['keywords'].values:
    list_of_sent.append(sent.split())

In [7]:

# removing html tags and apostrophes if present.
import re
def cleanhtml(sentence): #function to clean the word of any html-tags
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', sentence)
    return cleantext
def cleanpunc(sentence): #function to clean the word of any punctuation or special characters
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    return  cleaned

In [8]:
i=0
list_of_sent_train=[]
for sent in df['keywords'].values:
    filtered_sentence=[]
    sent=cleanhtml(sent)
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if(cleaned_words.isalpha()):    
                filtered_sentence.append(cleaned_words.lower())
            else:
                continue 
    list_of_sent_train.append(filtered_sentence)

In [9]:
import gensim
# Training the wor2vec model using train dataset
w2v_model=gensim.models.Word2Vec(list_of_sent_train,size=100, workers=4)

In [10]:
import numpy as np
sent_vectors = []; # the avg-w2v for each sentence/review is stored in this train
for sent in list_of_sent_train: # for each review/sentence
    sent_vec = np.zeros(100) # as word vectors are of zero length
    cnt_words =0; # num of words with a valid vector in the sentence/review
    for word in sent: # for each word in a review/sentence
        try:
            vec = w2v_model.wv[word]
            sent_vec += vec
            cnt_words += 1
        except:
            pass
    sent_vec /= cnt_words
    sent_vectors.append(sent_vec)
sent_vectors = np.array(sent_vectors)
sent_vectors = np.nan_to_num(sent_vectors)
sent_vectors.shape


(20, 100)

In [11]:
from sklearn.cluster import KMeans
model2 = KMeans(n_clusters=6, init='k-means++', n_init=7 , max_iter=300, random_state=11)
model2.fit(sent_vectors)

KMeans(n_clusters=6, n_init=7, random_state=11)

In [12]:
word_cluster_pred=model2.predict(sent_vectors)
word_cluster_pred_2=model2.labels_
word_cluster_center=model2.cluster_centers_

In [13]:
dfa = df
dfa['AVG-W2V Clus Label'] = model2.labels_
dfa.head(25)

,datasetName,keywords,AVG-W2V Clus Label
0,USA_cars_datasets,unnamed: 0 price brand model year titl statu m...,0
1,cars,manufactur name model name transmiss color odo...,3
2,test-data,unnamed: 0 name locat year kilomet driven fuel...,1
3,vehicles,id url region region url price year manufactur...,0
4,zoo,anim name hair feather egg milk airborn aquat ...,1
5,zoo2,anim name hair feather egg milk airborn aquat ...,1
6,zoo3,anim name hair feather egg milk airborn aquat ...,1
7,qs-world-university-rankings-2017-to-2022-V2,univers year rank display score link countri c...,0
8,cwurData,world rank institut countri nation rank qualit...,0
9,timesData,world rank univers name countri teach intern r...,0


In [14]:
from sklearn import metrics
silhouette_score = metrics.silhouette_score(sent_vectors, model2.labels_, metric='euclidean')

In [15]:
dfa.groupby(['AVG-W2V Clus Label'])['keywords'].count()

AVG-W2V Clus Label
0    12
1     4
2     1
3     1
4     1
5     1
Name: keywords, dtype: int64

In [16]:
silhouette_score

0.3249289313803677

In [17]:
from sklearn.metrics.cluster import rand_score
results = dfa['AVG-W2V Clus Label'].to_numpy()
actual = [0,0,0,0,1,1,1,2,2,2,3,3,3,4,4,4,4,4,5,5]
rand_score (actual,results)

0.6526315789473685